# Blotto

## Rules:
There are 10 castles, 2 players, each with 100 soldiers.
Simultaneously, each player allocates some number of soldiers to each castle.
For each castle, the player who allocated more soldiers to that castle "wins" that castle.
Castle `i` is worth `i` points. Ties evenly distribute points.
The goal may be to maximize probability of winning, or maximize expected number of points won.

In [1]:
import numpy as np

In [16]:
# in numpy batched operations, each row is a game.
# So a shape of an array may be (num_games, 10)
# Ideally, shape (10,) should be fine.
castle_points = np.arange(1, 11)


def get_points(a, b):
    # returns player a score.
    # Since sum of scores is 1 + ... + 10 = 55, easy to get b score
    wins = np.where(a > b, 1, 0) + np.where(a == b, 1 / 2, 0)
    return np.sum(wins * castle_points, axis=-1, keepdims=True)

## Simulation ideas:
The ultimate goal is to have the highest score or number of wins (to be determined) when matched against the pool of strategies submitted by other interns. This is not knowable. It is a good idea to brainstorm what strategies other interns may use. There may also be some database of strategies to compete against.

Ultimately, any Nash equilibrium for this game must be a mixed strategy. For a quick proof, consider a pure strategy. A strategy that beats this can be created by copying this strategy, increasing the soldiers allocated to castle 10, and decreasing the soldiers allocated to some other castle. The only way this process fails is if all soldiers are allocated to castle 10, and this is easy to beat.

But I/we cannot submit mixed strategies. 

A good question is, given a collection of pure strategies, come up with a pure strategy that maximizes the expected value of points or expected win rate against this collection. On the face, this seems hard: it appears to be a combinatorial optimization problem. Even integer programming is np hard, and our objective is nonlinear (although it is monotonic). We may be able to get somewhere by relaxing integrality and replacing the step function for the score with a sigmoid-esque function, allowing for gradient-based optimization.

Let's start with a genetic algorithm. We can make each organism a pure strategy or a mixed strategy. Let's start with pure.

In [146]:
def rank_population(population):  # population shape (pop_size, 10)
    """Takes population, returns array of shape (pop_size, 1) whose
    i-th entry is the index of the i-th best organism"""
    a = np.expand_dims(population, 1)  # shape (pop_size, 1, 10)
    b = np.expand_dims(population, 0)  # shape (1, pop_size, 10)
    points_by_game = get_points(a, b)
    total_points = np.sum(points_by_game, axis=1)  # shape (pop_size, 1)
    return np.argsort(total_points, axis=0)[::-1]  # sort descending by score

In [159]:
def mutate(population, gen, num_mutations=1):
    num_soldiers = np.sum(population, axis=-1)
    population = population.copy()
    # randomly move 1 soldier
    # choice to make: random by tower or random by soldier? I'll do by tower
    for _ in range(num_mutations):
        remove = gen.integers(population.shape[1], size=population.shape[0])
        add = gen.integers(population.shape[1], size=population.shape[0])
        idx = np.arange(population.shape[0])
        mutate_filter = population[idx, remove] > 0 # can't remove if 0 soldiers
        population[idx, remove] = np.where(
            mutate_filter,
            population[idx, remove] - 1,
            population[idx, remove]
        )
        population[idx, add] = np.where(
            mutate_filter, population[idx, add] + 1, population[idx, add]
        )
    assert np.all(np.sum(population, axis=-1) == num_soldiers) # assert soldiers preserved
    return population

In [160]:
def initialize(num_organisms, gen, num_castles=10, num_soldiers=100):
    population = np.zeros((num_organisms, num_castles), dtype=np.int8)
    for _ in range(num_soldiers):
        idx = np.arange(num_organisms)
        add = gen.integers(num_castles, size=num_organisms)
        population[idx, add] += 1
    return population

In [161]:
def genetic_step(population, gen, num_mutations=1):
    keep_ratio = 2 # keep 1/keep_ratio
    num_organisms = population.shape[0]
    rank = rank_population(population).reshape(-1) # shape (num_organisms)
    top_organisms = rank[:num_organisms//keep_ratio]
    top_organisms = population[top_organisms]
    new_population = np.tile(top_organisms, (keep_ratio, 1))
    return mutate(new_population, gen, num_mutations)

In [162]:
gen = np.random.default_rng(seed=1)
p = initialize(1000, gen, num_castles = 10, num_soldiers = 100)
for _ in range(100):
    p = genetic_step(p, gen, num_mutations=10)

In [163]:
p[:10, :]

array([[ 1,  7,  2, 10,  7,  2,  8, 29,  8, 26],
       [ 1,  5,  5, 11, 11,  0, 16,  8, 14, 29],
       [ 1,  5,  3,  8,  7,  6,  8, 25, 11, 26],
       [ 1,  6,  1,  9,  8,  3,  8, 28, 11, 25],
       [ 1,  8,  6,  8,  9,  1,  6, 27,  5, 29],
       [ 2,  0,  3,  4, 11,  6,  8, 25, 14, 27],
       [ 1,  2,  8,  3, 12,  2, 10, 24,  8, 30],
       [ 0,  8,  3,  6, 15,  2,  8, 24,  8, 26],
       [ 0,  1,  7,  4,  9,  6, 11, 23, 11, 28],
       [ 1,  1,  3,  3,  1, 21,  3, 26, 11, 30]], dtype=int8)

In [164]:
gen = np.random.default_rng(seed=2)
p = initialize(1000, gen, num_castles = 10, num_soldiers = 100)
for _ in range(200):
    p = genetic_step(p, gen, num_mutations=5)
p[:10, :]

array([[ 1,  3,  5,  3,  2, 17, 16,  4, 23, 26],
       [ 1,  5,  3,  4,  2, 11, 14,  8, 25, 27],
       [ 0,  4,  5,  4,  4,  8, 18,  8, 24, 25],
       [ 0,  3,  3,  5,  6,  8, 15,  9, 25, 26],
       [ 0,  2,  7,  1,  5,  8, 15,  9, 28, 25],
       [ 3,  0,  3,  8,  6,  1, 22, 15, 23, 19],
       [ 1,  1,  9,  1,  5,  9, 15,  6, 26, 27],
       [ 2,  1,  5,  4,  6,  6, 14, 10, 27, 25],
       [ 1,  5,  6,  4,  5,  5, 17,  6, 26, 25],
       [ 1,  5,  6,  6,  3, 12, 13,  3, 24, 27]], dtype=int8)

In [165]:
gen = np.random.default_rng(seed=3)
p = initialize(1000, gen, num_castles = 10, num_soldiers = 100)
num_mut = 11
for i in range(100):
    if i % 10 == 0:
        num_mut -= 1
    p = genetic_step(p, gen, num_mutations=num_mut)
p[:10, :]

array([[ 0,  5,  8,  1, 23,  3, 18, 10,  6, 26],
       [ 0,  4,  8,  1, 24,  3, 18, 11,  7, 24],
       [ 0,  1,  2, 13, 17,  4, 19,  6,  3, 35],
       [ 1,  1,  0, 13, 18,  4, 19,  6,  3, 35],
       [ 2,  4,  5,  2, 20,  1, 18, 10, 13, 25],
       [ 2,  6,  5,  1, 19,  3, 18,  8, 12, 26],
       [ 2,  4,  7,  0, 22,  3, 19,  8, 10, 25],
       [ 1,  5,  6,  0, 23,  3, 19,  8, 10, 25],
       [ 3,  5,  4,  1, 19,  2, 20, 10, 12, 24],
       [ 0,  0,  0, 13, 21,  5, 18,  6,  2, 35]], dtype=int8)